In [2]:
import pandas as pd
import numpy as np

# Sual-Cavab dataseti 

Suali Message 1 den oxsarliga gore secir ve uygun cavabi yeni message 2 ni verir

In [3]:
df1 = pd.read_csv(r"C:\Users\Lenovo\Desktop\Chemical_NLP\train.csv\train.csv")
df1

,role_1,topic;,sub_topic,message_1,message_2
0,Chemist_RoleType.ASSISTANT,Organic chemistry,Naming organic compounds,What is the IUPAC name for the organic compoun...,There can be several isomers with the molecula...
1,Chemist_RoleType.ASSISTANT,Organic chemistry,Naming organic compounds,"""What is the systematic name of the organic co...",The systematic name of the organic compound wi...
2,Chemist_RoleType.ASSISTANT,Organic chemistry,Naming organic compounds,What is the systematic name of the following c...,The systematic name of the compound CH3CH2CH2O...
3,Chemist_RoleType.ASSISTANT,Organic chemistry,Naming organic compounds,What is the IUPAC name of the following organi...,The IUPAC name of the given organic compound (...
4,Chemist_RoleType.ASSISTANT,Organic chemistry,Naming organic compounds,What is the systematic name of the organic com...,The systematic name of the organic compound wi...
...,...,...,...,...,...
19995,Chemist_RoleType.ASSISTANT,Chemical education,Difficulty in understanding the role of chemis...,How can the understanding of the role of chemi...,1. Public outreach and education: Organize wor...
19996,Chemist_RoleType.ASSISTANT,Chemical education,Difficulty in understanding the role of chemis...,"""How can the understanding of the role of chem...",To improve the understanding of the role of ch...
19997,Chemist_RoleType.ASSISTANT,Chemical education,Difficulty in understanding the role of chemis...,How can chemistry be utilized to promote susta...,Chemistry can be utilized to promote sustainab...
19998,Chemist_RoleType.ASSISTANT,Chemical education,Difficulty in understanding the role of chemis...,What are the specific economic benefits of stu...,Studying chemistry offers numerous economic be...


In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from sklearn.preprocessing import normalize
import faiss
import numpy as np
import pandas as pd

bi_encoder = SentenceTransformer('all-MiniLM-L6-v2')  
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')  
df1 = pd.read_csv(r"C:\Users\Lenovo\Desktop\Chemical_NLP\train.csv\train.csv")

corpus = df1['message_1'].tolist()
corpus_embeddings = bi_encoder.encode(corpus, convert_to_numpy=True, show_progress_bar=True)

corpus_embeddings_normalized = normalize(corpus_embeddings, norm='l2')  
dimension = corpus_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(dimension)  
faiss_index.add(corpus_embeddings_normalized)

def hybrid_search(query, top_k_faiss=100, top_k_final=1):
    query_embedding = bi_encoder.encode([query], convert_to_numpy=True)
    query_embedding = normalize(query_embedding, norm='l2')

    _, top_k_indices = faiss_index.search(query_embedding, top_k_faiss)
    top_k_indices = top_k_indices[0]

    cross_inp = [[query, corpus[idx]] for idx in top_k_indices]
    cross_scores = cross_encoder.predict(cross_inp)
    
    reranked = sorted(zip(top_k_indices, cross_scores), key=lambda x: x[1], reverse=True)

    print(f"\n🔎 Sorğu: {query}\n")
    for idx, score in reranked[:top_k_final]:
        print(f"📌 CrossEncoder Score: {score:.4f}")
        print(f"💬 Q: {df1.iloc[idx]['message_1']}")
        print(f"💡 A: {df1.iloc[idx]['message_2']}")
        print("-" * 80)


In [ ]:
hybrid_search("How can we name organic compounds?")

# Esas meqale dataseti 

Eger her hansisa xosagelmez answer yada basqa birsey olsa verilen input meqaleler oxsar secirlir

In [ ]:
df2 = pd.read_csv(r"C:\Users\Lenovo\Desktop\Chemical_NLP\American_chemical_vector")
df2

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder
from sklearn.preprocessing import normalize
import faiss
import pandas as pd
import numpy as np

df = df2[df2["text"].notna()].reset_index(drop=True)

texts = df["text"].tolist()
corpus_embeddings = bi_encoder.encode(texts, convert_to_numpy=True, show_progress_bar=True)
corpus_embeddings = normalize(corpus_embeddings, norm="l2")

dimension = corpus_embeddings.shape[1]
faiss_index = faiss.IndexFlatIP(dimension) 
faiss_index.add(corpus_embeddings)

def search_paper(query, top_k_faiss=50, top_k_final=2):
    query_embedding = bi_encoder.encode([query], convert_to_numpy=True)
    query_embedding = normalize(query_embedding, norm="l2")
    _, indices = faiss_index.search(query_embedding, top_k_faiss)
    top_indices = indices[0]
    pairs = [[query, texts[i]] for i in top_indices]
    scores = cross_encoder.predict(pairs)

    reranked = sorted(zip(top_indices, scores), key=lambda x: x[1], reverse=True)
    print(f"\n🔍 Sorğu: {query}\n")
    for i, (idx, score) in enumerate(reranked[:top_k_final], 1):
        row = df2.iloc[idx]
        print(f"📘 [{i}] Score: {score:.4f}")
        print(f"📝 Title: {row['title']}")
        print(f"👨‍🔬 Authors: {', '.join(row['authors']) if row['authors'] else 'N/A'}")
        print(f"📚 Journal: {row['journal']} | Year: {row['year']} | Type: {row['article_type']}")
        print(f"🗓️ Date: {row['pub_date']}")
        print(f"📄 Abstract: {row['abstract'] if pd.notna(row['abstract']) else '[Abstract yoxdur]'}")
        print("-" * 90)
